In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('retina')

In [2]:
from __future__ import print_function, division
import os,sys
import numpy as np
import torch # pytorch package, allows using GPUs
# fix seed
seed=17
np.random.seed(seed)
torch.manual_seed(seed)
from datetime import datetime
from glob import glob
import pandas as pd
import h5py

# Step 1. Identify data and info

In [3]:
X_train_file_info = pd.DataFrame({'filelist' : glob('data/tmax_train/*.h5')})

In [4]:
X_train_file_info['num'] = [int(file.split('/')[2].split('_')[0], 16) for file in X_train_file_info['filelist']]

In [5]:
X_train_file_info = X_train_file_info.sort_values('num')
X_train_file_info.index = range(len(X_train_file_info))

In [6]:
X_train_info = np.load('data/tmax_train/tmax_X_train_info.npz').get('arr_0')

In [7]:
y_train = np.load('data/tmax_train/tmax_y_train.npz', allow_pickle=True).get('arr_0')

# Step 2. Initialize the dataset with a data loader

In [20]:
import torch
from torch.utils.data import Dataset, DataLoader

class WeatherDataset(Dataset):
    def __init__(self, data_path):
        super().__init__()
        with h5py.File(data_path, 'r') as f:
            self.data_X_train = np.array(f['data'])
        

    def __getitem__(self, idx):
        return self.data_X_train[idx]

    def __len__(self):
        return len(self.data_X_train)

    
    
def load_data(dataset_path):
    # define dataset path


    # create dataset object
    dataset = WeatherDataset(dataset_path)

    # create data loader object
    batch_size = 1
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return data_loader

data_loader = load_data('data/tmax_train/0_train_i.h5')
    
    
    # file_info is a pandas dataframe with filepaths for each data record and corresponding indices
    
    


In [21]:
data_loader

In [22]:
for x, in data_loader:
    print('Dimensions of x:', x.shape)
    break

Dimensions of x: torch.Size([10, 11, 365])
